In [25]:
from data_util import load_embeddings, DMConfig, tfConfig
try:
    import _pickle as cPickle
except ImportError:
    import cPickle
import gzip
import os
import numpy as np
from defs import LBLS

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
config = DMConfig()
if os.path.exists(config.tok2id_path):
    with gzip.open(os.path.abspath(config.tok2id_path)) as f:
        tok2id = cPickle.load(f)
if os.path.exists(config.id2tok_path):
    with gzip.open(os.path.abspath(config.id2tok_path)) as f:
        id2tok = cPickle.load(f)

In [28]:
embeddings = load_embeddings(config, tok2id)

INFO:Initialized embeddings.


In [29]:
embeddings.shape

(10008, 50)

## Training

In [43]:
from tf_birnn_classifier import TfBiRNNClassifier
from defs import PROJECT_DIR

In [ ]:
model = TfBiRNNClassifier(embedding=embeddings, eta=0.001, use_attn = True,
                           ckpts_prefix='{}/checkpoints/linear_model/birnn-model'.format(PROJECT_DIR),
                           summaries_dir='{}/summaries-birnn/'.format(PROJECT_DIR)
                          )
model.fit(tfConfig, restore_weights=False, batches_to_eval=50, max_iter=1)

INFO:Initializing data manager...
INFO:took 0 s
INFO:Building model...
INFO:took 0 s


fw outputs:  (?, 300, 50)
bw outputs:  (?, 300, 50)
all outputs:  (?, 300, 100)
all states:  (?, 100)
W_attn: (100, 25)


INFO:-- Initializingt new model
INFO:Starting epoch 1
INFO: == Evaluating on development data after batch 50
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.628, Recall: 0.628, F1: 0.628
INFO:-- Saving model


## Evaluation

In [45]:
def print_example(pred, label, feats, length, email_id):
    ids_ = feats[:length][:, 0]
    print("==== {} ====".format(email_id))
    print("Predicted label: {}. True label: {}".format(LBLS[pred], LBLS[label]))
    print( " ".join((id2tok[i] for i in ids_)) )
    print("------------------------------------")

    

def print_examples(preds, outputs, inputs, lens, email_ids):
    for p, o, i, l, d in zip(preds, outputs, inputs, lens, email_ids):
        print_example(p, o, i, l, d)

In [46]:
preds, inputs, lens, outputs, email_ids = model.predict()
correct = preds == outputs
print("Predicted {} of {} examples correctly (accuracy: {:.3f})".format(np.sum(correct), correct.shape[0], np.mean(correct)))

Predicted 19 of 32 examples correctly (accuracy: 0.594)


In [47]:
print_examples(preds, outputs, inputs, lens, email_ids)

==== b'<14438841.1075848147472.JavaMail.evans@thyme>' ====
Predicted label: NORESPONSE. True label: REPLY
<s> attached are the presentations for the corporate staff group and the NNNUMMM goals . there have been slight modifications to the format . </s>
------------------------------------
==== b'<16926152.1075862380054.JavaMail.evans@thyme>' ====
Predicted label: NORESPONSE. True label: NORESPONSE
<s> john sherriff and michael brown would like to send out the attached letter to employees in europe this afternoon ( ie in the next NNNUMMM hours ) . i would be grateful if you would review it and let us have your approval as soon as possible . i have a couple of questions in relation to the content : NNNUMMM . can we refer to wessex water on our list of non-core assets in europe ? NNNUMMM . can we list the specific businesses under review as per the proposed draft below ( which is not included in the attached draft ? ) `` the businesses currently under review in europe are global markets (